In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

e:\app\python3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
e:\app\python3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_cols = train.columns.difference(['id', 'label'])
train_cols

Index(['FIELD_1', 'FIELD_10', 'FIELD_11', 'FIELD_12', 'FIELD_13', 'FIELD_14',
       'FIELD_15', 'FIELD_16', 'FIELD_17', 'FIELD_18', 'FIELD_19', 'FIELD_2',
       'FIELD_20', 'FIELD_21', 'FIELD_22', 'FIELD_23', 'FIELD_24', 'FIELD_25',
       'FIELD_26', 'FIELD_27', 'FIELD_28', 'FIELD_29', 'FIELD_3', 'FIELD_30',
       'FIELD_31', 'FIELD_32', 'FIELD_33', 'FIELD_34', 'FIELD_35', 'FIELD_36',
       'FIELD_37', 'FIELD_38', 'FIELD_39', 'FIELD_4', 'FIELD_40', 'FIELD_41',
       'FIELD_42', 'FIELD_43', 'FIELD_44', 'FIELD_45', 'FIELD_46', 'FIELD_47',
       'FIELD_48', 'FIELD_49', 'FIELD_5', 'FIELD_50', 'FIELD_51', 'FIELD_52',
       'FIELD_53', 'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57', 'FIELD_6',
       'FIELD_7', 'FIELD_8', 'FIELD_9', 'age_source1', 'age_source2',
       'district', 'maCv', 'province'],
      dtype='object')

In [4]:
numberic_cols = ['FIELD_3','FIELD_50','FIELD_51','FIELD_52','FIELD_53', 'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57']
train[numberic_cols] = train[numberic_cols].replace(['None', 'nan', 'na', 'NaN'], -1)
train[numberic_cols] = train[numberic_cols].fillna(-99)
test[numberic_cols] = test[numberic_cols].replace(['None', 'nan', 'na', 'NaN'], -1)
test[numberic_cols] = test[numberic_cols].fillna(-99)

In [5]:
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [6]:
final_iv, IV = data_vars(train,train.label)

e:\app\python3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
e:\app\python3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in log
e:\app\python3\lib\site-packages\ipykernel_launcher.py:94: RuntimeWarning: divide by zero encountered in log
e:\app\python3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
e:\app\python3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in log
e:\app\

In [7]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,id,0,14999,15000,238.0,0.015867,14762.0,0.984133,0.489712,0.500169,-0.021130,0.000437
1,id,15000,29999,15000,248.0,0.016533,14752.0,0.983467,0.510288,0.499831,0.020706,0.000437
2,province,Thành phố Cần Thơ,Thành phố Cần Thơ,192,2.0,0.010417,190.0,0.989583,0.004115,0.006438,-0.447466,0.138674
3,province,Thành phố Hà Nội,Thành phố Hà Nội,667,10.0,0.014993,657.0,0.985007,0.020576,0.022261,-0.078688,0.138674
4,province,Thành phố Hải Phòng,Thành phố Hải Phòng,252,0.0,0.000000,252.0,1.000000,0.000000,0.008538,0.000000,0.138674
5,province,Thành phố Hồ Chí Minh,Thành phố Hồ Chí Minh,2389,36.0,0.015069,2353.0,0.984931,0.074074,0.079725,-0.073516,0.138674
6,province,Thành phố Đà Nẵng,Thành phố Đà Nẵng,228,2.0,0.008772,226.0,0.991228,0.004115,0.007657,-0.620976,0.138674
7,province,Tỉnh An Giang,Tỉnh An Giang,608,11.0,0.018092,597.0,0.981908,0.022634,0.020228,0.112390,0.138674
8,province,Tỉnh Bà Rịa - Vũng Tàu,Tỉnh Bà Rịa - Vũng Tàu,306,6.0,0.019608,300.0,0.980392,0.012346,0.010165,0.194388,0.138674
9,province,Tỉnh Bình Dương,Tỉnh Bình Dương,807,9.0,0.011152,798.0,0.988848,0.018519,0.027038,-0.378473,0.138674


In [8]:
# IV.sort_values('IV')

In [9]:
transform_vars_list = train.columns.difference(['label', 'id', 'province', 'district', 'maCv', 'FIELD_7'])
transform_prefix = 'new_' # leave this value blank if you need replace the original column values

In [10]:
transform_vars_list

Index(['FIELD_1', 'FIELD_10', 'FIELD_11', 'FIELD_12', 'FIELD_13', 'FIELD_14',
       'FIELD_15', 'FIELD_16', 'FIELD_17', 'FIELD_18', 'FIELD_19', 'FIELD_2',
       'FIELD_20', 'FIELD_21', 'FIELD_22', 'FIELD_23', 'FIELD_24', 'FIELD_25',
       'FIELD_26', 'FIELD_27', 'FIELD_28', 'FIELD_29', 'FIELD_3', 'FIELD_30',
       'FIELD_31', 'FIELD_32', 'FIELD_33', 'FIELD_34', 'FIELD_35', 'FIELD_36',
       'FIELD_37', 'FIELD_38', 'FIELD_39', 'FIELD_4', 'FIELD_40', 'FIELD_41',
       'FIELD_42', 'FIELD_43', 'FIELD_44', 'FIELD_45', 'FIELD_46', 'FIELD_47',
       'FIELD_48', 'FIELD_49', 'FIELD_5', 'FIELD_50', 'FIELD_51', 'FIELD_52',
       'FIELD_53', 'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57', 'FIELD_6',
       'FIELD_8', 'FIELD_9', 'age_source1', 'age_source2'],
      dtype='object')

In [11]:
# for var in transform_vars_list:
#     try:
#         small_df = final_iv[final_iv['VAR_NAME'] == var]
#         transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
#         replace_cmd = ''
#         replace_cmd1 = ''
#         for i in (transform_dict.items()):
#             replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
#             replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
#         replace_cmd = replace_cmd + '0'
#         replace_cmd1 = replace_cmd1 + '0'
#         if replace_cmd != '0':
#             try:
#                 train[transform_prefix + var] = train[var].apply(lambda x: eval(replace_cmd))
#             except:
#                 train[transform_prefix + var] = train[var].apply(lambda x: eval(replace_cmd1))
#     except:
#         print(var)

In [12]:
small_df = final_iv[final_iv['VAR_NAME'] == 'FIELD_1']
transform_range = zip(small_df.MIN_VALUE, small_df.MAX_VALUE,small_df.WOE)
for h, k, v in transform_range:
    print(h, k, v)

0 0 -0.3154367467376522
1 1 0.07588310601797468


In [13]:
numberic_cols = ['FIELD_3','FIELD_50','FIELD_51','FIELD_52','FIELD_53', 'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57']

In [14]:
for var in transform_vars_list:
    small_df = final_iv[final_iv['VAR_NAME'] == var]
    if var in numberic_cols:
        print('WOE extract with numberic type ' + var)
        transform_range = zip(small_df.MIN_VALUE, small_df.MAX_VALUE,small_df.WOE)
        def apply_func(x):
            for h, k, v in transform_range:
                print (h, k, v)
                if x > float(h) and x <= float(k) :
                    return v
            return 0
        train[transform_prefix + var] = train[var].apply(lambda x : apply_func(x))
        test[transform_prefix + var] = test[var].apply(lambda x : apply_func(x))
    else:
        print('WOE extract with object type ' + var)
        transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
        train[transform_prefix + var] = train[var].map(transform_dict)
        test[transform_prefix + var] = test[var].map(transform_dict)

WOE extract with object type FIELD_1
WOE extract with object type FIELD_10
WOE extract with object type FIELD_11
WOE extract with object type FIELD_12
WOE extract with object type FIELD_13
WOE extract with object type FIELD_14
WOE extract with object type FIELD_15
WOE extract with object type FIELD_16
WOE extract with object type FIELD_17
WOE extract with object type FIELD_18
WOE extract with object type FIELD_19
WOE extract with object type FIELD_2
WOE extract with object type FIELD_20
WOE extract with object type FIELD_21
WOE extract with object type FIELD_22
WOE extract with object type FIELD_23
WOE extract with object type FIELD_24
WOE extract with object type FIELD_25
WOE extract with object type FIELD_26
WOE extract with object type FIELD_27
WOE extract with object type FIELD_28
WOE extract with object type FIELD_29
WOE extract with numberic type FIELD_3
-99.0 -1.0 0.1714370028720519
337.0 711.0 0.07873320720540813
712.0 1448.0 0.07381853013119215
1449.0 2545.0 -0.007735807395379

In [15]:
train[[col for col in train.columns if col.startswith('new_')]].to_csv('..\input\woe_train.csv', index=False)
test[[col for col in train.columns if col.startswith('new_')]].to_csv('..\input\woe_test.csv', index=False)